### IR Wavefunction

$$
\newcommand{\ket}[1]{\vert{#1}\rangle}\\
\newcommand{\bra}[1]{\langle{#1}\vert}\\
\ket{\text{IR}} = \ket{\hat n_{k_{-2}} = \hat n_{k_{-3}} = 2} \otimes \frac{1}{2}\bigg[ \ket{\uparrow^d}\frac{1}{\sqrt 2}\left(\ket{k_1 \downarrow} + \ket{k_{-1} \downarrow}\right) - \ket{\downarrow^d}\frac{1}{\sqrt 2}\left(\ket{k_1 \uparrow} + \ket{k_{-1} \uparrow}\right) + \ket{2^d}\ket{0} - \ket{0^d}\ket{k_1 \uparrow, k_1 \downarrow,k_{-1} \uparrow,k_{-1} \downarrow}\bigg] \otimes  \ket{\hat n_{k_{2}} = \hat n_{k_{3} } = 0} = \ket{1111}\ket{0000}\\
=\frac{1}{2}\ket{0000} \otimes \left[\ket{10}\ket{0101} - \ket{01}\ket{1010} + \ket{11}\ket{0000} - \ket{00}\ket{1111}\right] \otimes \ket{1111}\\
U^\dagger_{k_+, k_-} = \frac{1}{2}\left[1 + \eta_{k_- \uparrow} + \eta^\dagger_{k_+ \uparrow} + \eta_{k_- \downarrow} + \eta^\dagger_{k_+ \downarrow}\right] \\
 \\
\eta_{k_+\beta}^\dagger = V \left[\lambda_2 \hat n_{d\overline\beta} + \lambda_1 \left( 1 - \hat n_{d \overline \beta} \right)  \right]c^\dagger_{q\beta}c_{d\beta} + \lambda_3 \sum_{k} \left\{ \frac{J}{2}\left( S_d^z \beta c^\dagger_{q\beta}c_{k\beta} + c^\dagger_{d \overline \beta}c_{d\beta}c^\dagger_{q\beta}c_{k \overline \beta}\right) + \frac{K}{2}\left( C^z_d c^\dagger_{q\beta}c_{k\beta} + c^\dagger_{q\beta}c^\dagger_{k \overline \beta}c_{d \overline \beta}c_{d\beta} \right) \right\}\\
\eta_{k_-\beta} = V^* \left[\lambda_1 \hat n_{d \overline \beta} + \lambda_2 \left( 1 - \hat n_{d \overline \beta} \right)  \right]c^\dagger_{d\beta}c_{q\beta} + \lambda_3 \sum_{k} \left\{ \frac{J}{2}\left( S_d^z \beta c^\dagger_{k\beta}c_{q\beta} + c^\dagger_{d\beta}c_{d \overline \beta}c^\dagger_{k \overline \beta}c_{q\beta}\right)+ \frac{K}{2}\left( C^z_d c^\dagger_{k\beta}c_{q\beta} + c^\dagger_{d\beta}c^\dagger_{d \overline \beta}c_{k \overline \beta}c_{q\beta} \right) \right\} \\
\lambda_1 = \frac{1}{\omega - \frac{1}{2}D + \epsilon_d + \frac{1}{2}K}, \lambda_2 = \frac{1}{\omega - \frac{1}{2}D - \epsilon_d + \frac{1}{2}J}, \lambda_3 = \frac{1}{\omega - \frac{1}{2}D + \frac{1}{4}\left(J + K\right)} 
$$

### Creating impurity and c-bath basis

In [59]:
from qutip import *
from math import *
import itertools
import numpy as np
from itertools import product

### Writing the IR wavefunction

In [60]:
def get_cloud(fock):
    return tensor([basis(2,i) for i in fock])


out = 1
# no of momenta states outside cloud, on each side of FS

ins = 1
# no of momenta states inside cloud, on each side of FS

iom_above = tensor([basis(2,0)] * 2 * out)
iom_below = tensor([basis(2,1)] * 2 * out)
cloud1 = get_cloud([1, 0, 0, 1, 0, 1])
cloud2 = get_cloud([0, 1, 1, 0, 1, 0])
cloud3 = get_cloud([1, 1, 0, 0, 0, 0])
cloud4 = get_cloud([0, 0, 1, 1, 1, 1])
psi_IR = tensor(iom_above, cloud1 - cloud2 + cloud3 - cloud4, iom_below).unit()

## Creating the unitary

$ \eta^{\dagger/-} = \eta_1^{\dagger/-} + \eta_2^{\dagger/-} + \eta_3^{\dagger/-}$

### Create $c^\dagger_{q\beta}\otimes I_{\overline\beta}$ or $c_{q\beta}\otimes I_{\overline\beta}$

In [61]:
def cq(beta, conj):
    cq_ = create(2) if conj == 1 else destroy(2)
    # use c^dagger_q is conj == 1 else c_q

    return tensor(cq_, idt) if beta == 1 else tensor(idt, cq_)
    # place cq first if beta == up else place idt first

### Creating $\sum_k c_{k\beta}$

In [62]:
def sum_k_ck(beta, conj):
    # returns \sum_{k}c_{k\beta}, the conjugate is obtained by applying .dag() on this return value

    ck = create(2) if conj == 1 else destroy(2)
    # variable conj is supposed to tell me whether i require \sum_k c^\dagger_k or \sum_k c_k

    sum=0
    num = 2 * ins
    for i in range(num):
        # outer loop shifts the position of k over all cloud momenta

        cur = 0
        for j in range(num):
            # inner loop creates the string for a particular k

            if i != j: # if not the particular k, fill with idt
                cur = tensor(cur, idt, idt) if cur !=0 else tensor(idt, idt)
            elif beta == 1: 
                # otherwise, check if spin is up or down, up goes first, down goes second
                cur = tensor(cur, ck, idt) if cur !=0 else tensor(ck, idt)
            else: 
                cur = tensor(cur, idt, ck) if cur !=0 else tensor(idt, ck)
        sum = cur if sum == 0 else sum + cur 
    return sum

### Creating first part of $\eta$

$$
V \left[\lambda_2 \hat n_{d\overline\beta} + \lambda_1 \left( 1 - \hat n_{d \overline \beta} \right)  \right]c^\dagger_{q\beta}c_{d\beta}
$$
or
$$
V^* \left[\lambda_1 \hat n_{d \overline \beta} + \lambda_2 \left( 1 - \hat n_{d \overline \beta} \right)  \right]c^\dagger_{d\beta}c_{q\beta}
$$

In [63]:
def eta1(beta, conj):
    cimp = create(2) if conj == -1 else destroy(2)
    # impurity beta operator will be c_{d\beta} if conj = 1 else c^\dagger_{d\beta}
    
    imp_part = [cimp, c2 * n + c1 * (1-n)] if beta == 1 else [c2 * n + c1 * (1-n), cimp]
    # the full impurity part will be cimp \otimes idt(2) if beta is up, else idt(2) \otimes cimp,
    # because up part comes first

    cloud_part = [idt]*2*2*ins
    # cloud electrons do not participate in this part of eta, hence just identities. 
    # (ins) is number of k states on one side of FS, *2 for both sides, another *2 for spin 

    eta_1 = V * tensor([c_q] + imp_part + cloud_part) if conj == 1 else V * tensor(imp_part + cloud_part + [c_q])
    # if conj == 1 (if we are creating eta^dagger), then q is above the FS and eta_1 will be 
    # c^\dagger_q \otimes imp_part \otimes cloud_part, otherwise it will be imp_part \otimes cloud_part \otimes c_q,
    # because q will be below the FS

    return eta_1

### Creating second part of $\eta$

$$
\lambda_3 \sum_{k} \frac{J}{2}c^\dagger_{q\beta}\left( S_d^z \beta c_{k\beta} + c^\dagger_{d \overline \beta}c_{d\beta}c_{k \overline \beta}\right), \text{ conj} == 1
$$
or
$$
\lambda_3 \sum_{k} \frac{J}{2}\left( S_d^z \beta c^\dagger_{k\beta} + c^\dagger_{d\beta}c_{d \overline \beta}c^\dagger_{k \overline \beta}\right)c_{q\beta}, \text{ conj} == -1
$$
The impurity part can be written as
$$
\text{imp-part} = \begin{cases} c^\dagger_{d\uparrow}c_{d \downarrow}, \text{ if }\beta \neq \text{conj} \\ -c_{d\uparrow}c^\dagger_{d \downarrow}, \text{ if }\beta == \text{conj} \end{cases}
$$
Cloud part is $\sum_k c_{k\beta}(conj)$

In [64]:
def eta2(beta, conj):
    imp_part = -tensor(destroy(2), create(2)) if conj == beta else tensor(create(2), destroy(2))
    
    k_beta = sum_k_ck(beta, conj)
    k_beta_bar = sum_k_ck(-beta, conj)

    eta_2 = (tensor(c_q, betaSdz, k_beta) + tensor(c_q, imp_part, k_beta_bar)) if conj == 1 else (tensor(betaSdz, k_beta, c_q) + tensor(imp_part, k_beta_bar, c_q))
    return (c3 / 2) * J * eta_2

### Creating third part of $\eta$

$$
\lambda_3 \sum_{k} \frac{K}{2}\left( C^z_d c^\dagger_{q\beta}c_{k\beta} + c^\dagger_{q\beta}c^\dagger_{k \overline \beta}c_{d \overline \beta}c_{d\beta} \right)
$$
or
$$
\lambda_3 \sum_{k} \frac{K}{2}\left( C^z_d c^\dagger_{k\beta}c_{q\beta} + c^\dagger_{d\beta}c^\dagger_{d \overline \beta}c_{k \overline \beta}c_{q\beta} \right)
$$
Impurity part is
$$
\text{imp-part} = \begin{cases}-c_{d \uparrow}c_{d \downarrow}, \beta = \text{conj} = 1 \\
c_{d \uparrow}c_{d \downarrow}, \beta = -1, \text{conj} = 1 \\
c^\dagger_{d \uparrow}c^\dagger_{d \downarrow}, \beta = 1, \text{conj} = -1 \\
-c^\dagger_{d \uparrow}c^\dagger_{d \downarrow}, \beta = \text{conj} = -1 \end{cases} = -(\beta \times \text{conj}) * (c_{d \uparrow}c_{d \downarrow})(conj)
$$
The cloud parts are
$$
\sum_k(\beta, -conj) \text{ and }\sum_k(-\beta, conj)
$$

In [65]:
def eta3(beta, conj=True):
    imp_part = -beta*conj*(tensor(destroy(2), destroy(2)) if conj == 1 else tensor(create(2), create(2)))
    
    k_beta = sum_k_ck(beta, -conj)
    k_beta_bar = sum_k_ck(-beta, conj)

    eta_3 = (tensor(c_q, Cdz, k_beta) + tensor(c_q, imp_part, k_beta_bar)) if conj == 1 else (tensor(Cdz, k_beta, c_q) + tensor(imp_part, k_beta_bar, c_q))
    return (c3 / 2) * K * eta_3

In [66]:
c1 = c2 = c3 = V = J = K = 1

idt = identity(2)
n = create(2) * destroy(2)
# shorthands for identity and number operators

Cdz = 0.5 * (tensor(n, idt) +  tensor(idt, n) - 1)
# isospin z component

psi = psi_IR
for jout in range(out):
    Udag = 1/2
    for conj, beta in product([1, -1], [1, -1]):    
        # conj = 1 means eta dagger (q is above FS), conj = -1 means eta (q is below)

        print (conj, beta)
        iom1 = 2*(out - 1) if conj == 1 else 2 * out
        iom2 = 2*(out - 1) if conj == -1 else 2 * out
        # iom1 == k states above the FS that are unchanged b4 & after this reverse RG step.
        # If conj == 1, that means I am working with a q that's empty, so iom1 will lose 2 states (q up, q down).
        # iom2 will lose two states if the electron being re-entangled is below FS, that is if conj == -1

        c_q = cq(beta, conj)
        # gets either c^\dagger_{q\beta}\otimes identity(q\ol\beta), or c_{q\beta}iden(q\ol\beta), depending on conj=\pm 1

        betaSdz = beta * 0.5 * (tensor(n, idt) - tensor(idt, n))
        # gets \beta S_d^z

        eta_1 = eta1(beta, conj)
        eta_2 = eta2(beta, conj)
        eta_3 = eta3(beta, conj)
        # get the three parts of eta (V part, S part and C part)
        eta_beta_conj = eta_1 + eta_2 + eta_3

        for k in range(iom1):
            eta_beta_conj = tensor(idt, eta_beta_conj)
        for k in range(iom2):
            eta_beta_conj = tensor(eta_beta_conj, idt)


        # get eta for this value of beta and conf. For example, this is the eta for k=+1 and beta = up

        Udag += eta_beta_conj/2
    
    print (psi)
    psi = Udag(psi)
    out -= 1
    ins += 1



1 1
1 -1
-1 1
-1 -1
Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], shape = (1024, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
